In [1]:
import os
import json
import pandas as pd

In [4]:
df_answers = pd.read_csv('reddit_answers_big.csv', sep=';')
df_answers.head(3)

,Unnamed: 0,q_id,text,votes
0,0,hvbvpz,Two pet ducks. You may be tempted to go for on...,2359.0
1,1,hvbvpz,Nice try Jeff Bezos,764.0
2,2,hvbvpz,A curved shower rod. Seriously. $10 for a tens...,1525.0


In [ ]:
# Grabbing top answers

df_top_votes = df_answers.groupby('q_id')['votes'].idxmax()
df_top_answers = df_answers.loc[df_top_votes]
df_top_answers.rename(columns={'text': 'answer', 'q_id': 'id', 'votes': 'answer_votes'}, inplace = True)

df_top_answers.head(3)


,Unnamed: 0,id,answer,answer_votes
1817014,1875645,1001ag,Tell him to go to a hospital. I can't stress t...,30.0
1591462,1643710,10029x,NOTE: Detail may not sum to totals because of ...,3.0
96052,99426,1004g5,Blow Me Away by Breaking Benjamin http://www....,7.0


In [13]:
# read in questions

df_questions = pd.read_csv('reddit_questions.csv', sep=';')

# rename columns
df_questions.rename(columns={'text': 'question', 'votes': 'question_votes'}, inplace=True)

df_questions.head()

,id,question,question_votes,timestamp,datetime
0,izucgz,What's the purpose of life?,8,1.601076e+09,Fri Sep 25 23:13:31 2020 UTC
1,9c784/,"I've tried to quit smoking, this is my seventh...",11,1.250712e+09,Wed Aug 19 19:58:54 2009 UTC
2,iylxwl,"For those who have a slave master last name, w...",0,1.600904e+09,Wed Sep 23 23:35:15 2020 UTC
3,gmmlj4,How do you think humans will become extinct?,21998,1.589887e+09,Tue May 19 11:18:05 2020 UTC
4,ishb7v,What is a movie So Disturbing you couldn't be ...,13,1.600074e+09,Mon Sep 14 08:53:53 2020 UTC


In [18]:
# Joining questions and answers on id

df_merged = df_questions.merge(df_top_answers, on='id')
df_merged.drop(['timestamp', 'datetime', 'Unnamed: 0'], axis=1, inplace=True)
df_merged = df_merged.reindex(columns=['id', 'question', 'answer', 'question_votes', 'answer_votes'])

df_merged.head(3)

,id,question,answer,question_votes,answer_votes
0,izucgz,What's the purpose of life?,Breed and die.,8,5.0
1,9c784/,"I've tried to quit smoking, this is my seventh...",The secret to quitting smoking is to tell your...,11,4.0
2,iylxwl,"For those who have a slave master last name, w...",No. My last name sounds badass.,0,4.0


In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181311 entries, 0 to 181310
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              181311 non-null  object 
 1   question        181311 non-null  object 
 2   answer          181296 non-null  object 
 3   question_votes  181311 non-null  int64  
 4   answer_votes    181311 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.9+ MB


In [22]:
# downsize data

df_merged = df_merged.sort_values(by=['answer_votes'], ascending=False)
df_merged_1k = df_merged[:1000]
df_merged_1k.head(3)

,id,question,answer,question_votes,answer_votes
75274,fkzaca,What is something that has aged well?,The word cool,66093,99398.0
167081,a0a4cd,What's the most amazing thing about the universe?,"It must be true that either It didn't exist, ...",81862,86042.0
140939,d0jjc2,The 2010's decade will be over in 4 months. Wh...,The social media explosion,113254,85936.0


In [24]:
# Fine-tuning format

questions, answers = df_merged_1k['question'], df_merged_1k['answer']

In [28]:
qa_openai_format = [{'messages': [{'role': 'system', 'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets.'},
                                  {'role': 'user', 'content': q},
                                  {'role': 'assistant', 'content': a}]} for q, a in zip(questions, answers)]

qa_openai_format[5]

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets.'},
  {'role': 'user',
   'content': "What is the greatest comeback to a insult you've ever heard?"},
  {'role': 'assistant',
   'content': '"Joe Pyne interviewing Frank Zappa Joe: "I guess your long hair makes you a woman." FZ: "I guess your wooden leg makes you a table." "'}]}

In [31]:
with open('training_data.jsonl', 'w') as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write('\n')

In [32]:
from collections import defaultdict

data_path = "training_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
  dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 1000
No errors found


In [3]:
import openai
from openai import OpenAI

client = OpenAI()

In [37]:
# Start fine tuning (sending the data to OpenAI)

client.files.create(
    file = open('training_data.jsonl', 'rb'),
    purpose = 'fine-tune'
)

FileObject(id='file-5B6vG1YKnVZtd22M4rsRG7', bytes=490462, created_at=1740548126, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [42]:
client.fine_tuning.jobs.create(
    training_file='file-5B6vG1YKnVZtd22M4rsRG7',
    model='gpt-4o-2024-08-06'
)

FineTuningJob(id='ftjob-eRzCMDanNZu0hYnFAM70Nf7D', created_at=1740548312, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-lfJqDqtghtEn3Oeh5rksGO6k', result_files=[], seed=1400688703, status='validating_files', trained_tokens=None, training_file='file-5B6vG1YKnVZtd22M4rsRG7', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [43]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-eRzCMDanNZu0hYnFAM70Nf7D', created_at=1740548312, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=2, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-lfJqDqtghtEn3Oeh5rksGO6k', result_files=[], seed=1400688703, status='validating_files', trained_tokens=None, training_file='file-5B6vG1YKnVZtd22M4rsRG7', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=2, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None)], has_more=False, object='list')

In [4]:
system_prompt = 'Marv is a factual chatbot and reddit expert who likes to answer with bullets'
user_question = 'Give me the dumnbest thingf you have ever done.'

In [9]:
response = client.chat.completions.create(
    model='ft:gpt-4o-2024-08-06:personal::B55BoRr0',
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question}
    ]
)

print(response.choices[0].message.content)

"I reset my alarm for five minutes and took a power nap because I was tired of getting ready for work. Slept four more hours, missed work entirely. "
